In [1]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
h2o.init()
h2o.cluster().show_status()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 hour 54 mins
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.10
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_rishav_weluzh
H2O cluster total nodes:,1
H2O cluster free memory:,1.473 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


H2O cluster uptime:,1 hour 54 mins
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.10
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_rishav_weluzh
H2O cluster total nodes:,1
H2O cluster free memory:,1.473 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [2]:
df = h2o.import_file("Claims1.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
cols = list(df.columns)
cols.pop(cols.index('FraudFound_P'))
df = df[cols+['FraudFound_P']]

In [4]:
df[32] = df[32].asfactor()
predictors = df.columns[0:32]
response = 'FraudFound_P'

In [5]:
df.head()

WeekOfMonth,WeekOfMonthClaimed,Age,PolicyNumber,RepNumber,Deductible,DriverRating,Year,Month,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,Sex,MaritalStatus,Fault,PolicyType,VehicleCategory,VehiclePrice,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,BasePolicy,FraudFound_P
5,1,21,1,12,300,1,1994,11,2,6,1,1,0,0,0,0,5,1,5,4,3,0,2,3,1,0,1,0,1,2,2,0
3,4,34,2,15,400,4,1994,0,2,6,1,0,0,1,0,0,4,1,5,4,3,0,5,4,0,0,1,0,0,0,1,0
5,2,47,3,7,400,3,1994,9,4,6,1,3,10,1,1,0,4,1,5,4,3,1,6,6,1,0,1,0,0,0,1,0
2,1,65,4,4,400,2,1994,6,5,17,0,4,6,1,1,1,2,1,1,4,3,1,7,7,0,0,1,3,0,0,2,0
5,2,27,5,3,400,1,1994,0,0,6,1,1,1,0,0,1,4,1,5,4,3,0,4,4,1,0,1,0,0,0,1,0
4,1,20,6,12,400,3,1994,9,4,6,1,2,10,1,0,1,4,1,5,4,3,0,4,2,1,0,1,2,0,0,1,0
1,3,36,7,14,400,1,1994,1,5,6,1,0,1,1,1,1,4,1,5,4,3,1,6,5,1,0,1,1,0,0,1,0
1,4,0,8,1,400,4,1994,10,4,6,1,1,2,1,0,0,4,1,5,4,3,1,0,0,1,0,1,0,0,0,1,0
4,5,30,9,7,400,4,1994,11,5,6,1,2,11,1,0,0,4,1,5,4,3,0,5,4,1,1,1,2,0,0,1,0
3,3,42,10,7,400,1,1994,3,1,5,1,2,3,1,1,0,6,2,5,4,3,2,7,5,1,0,1,2,0,0,0,0


In [6]:
train, test, valid = df.split_frame(ratios=[.9, .09], seed = 1234)

In [7]:
rf = H2ORandomForestEstimator(balance_classes = True, seed = 1234)
rf.train(x = predictors, y = response, training_frame = train, validation_frame = valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [8]:
rf.confusion_matrix()

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19937920848722196: 


,0,1,Error,Rate
0,13036.0,6.0,0.0005,(6.0/13042.0)
1,2.0,13054.0,0.0002,(2.0/13056.0)
Total,13038.0,13060.0,0.0003,(8.0/26098.0)


In [9]:
rf.model_performance(test_data = test)


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.0508821652121906
RMSE: 0.22557075433706072
LogLoss: 0.2474722505812874
Mean Per-Class Error: 0.19373676217879554
AUC: 0.8658783918794709
Gini: 0.7317567837589418
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.03164475931368697: 


,0,1,Error,Rate
0,1235.0,82.0,0.0623,(82.0/1317.0)
1,39.0,37.0,0.5132,(39.0/76.0)
Total,1274.0,119.0,0.0869,(121.0/1393.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0316448,0.3794872,100.0
max f2,0.0214883,0.5057252,170.0
max f0point5,0.0468541,0.3993056,48.0
max accuracy,0.1739915,0.9454415,1.0
max precision,0.1739915,0.5,1.0
max recall,0.0,1.0,399.0
max specificity,0.1862247,0.9992407,0.0
max absolute_mcc,0.0214883,0.3553403,170.0
max min_per_class_accuracy,0.0140007,0.7760061,246.0
max mean_per_class_accuracy,0.0096908,0.8062632,298.0


Gains/Lift Table: Avg response rate:  5.46 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100503,0.0772148,7.8552632,7.8552632,0.4285714,0.4285714,0.0789474,0.0789474,685.5263158,685.5263158
,2,0.0201005,0.0618278,9.1644737,8.5098684,0.5,0.4642857,0.0921053,0.1710526,816.4473684,750.9868421
,3,0.0301508,0.0520119,7.8552632,8.2916667,0.4285714,0.4523810,0.0789474,0.25,685.5263158,729.1666667
,4,0.0402010,0.0450110,5.2368421,7.5279605,0.2857143,0.4107143,0.0526316,0.3026316,423.6842105,652.7960526
,5,0.0502513,0.0430932,5.2368421,7.0697368,0.2857143,0.3857143,0.0526316,0.3552632,423.6842105,606.9736842
,6,0.1005025,0.0294273,3.1421053,5.1059211,0.1714286,0.2785714,0.1578947,0.5131579,214.2105263,410.5921053
,7,0.1500359,0.0224464,2.9220061,4.3849156,0.1594203,0.2392344,0.1447368,0.6578947,192.2006102,338.4915638
,8,0.2002872,0.0177140,1.3092105,3.6132334,0.0714286,0.1971326,0.0657895,0.7236842,30.9210526,261.3233352
,9,0.3000718,0.0115361,1.4504922,2.8940443,0.0791367,0.1578947,0.1447368,0.8684211,45.0492238,189.4044321
,10,0.3998564,0.0054375,0.6593147,2.3363649,0.0359712,0.1274686,0.0657895,0.9342105,-34.0685346,133.6364925


In [10]:
rf_v2 = H2ORandomForestEstimator(
    model_id="rf_covType_v2",
    ntrees=200,
    max_depth=30,
    stopping_rounds=2,
    stopping_tolerance=0.01,
    score_each_iteration=True,
    seed=3000000)

In [11]:
rf_v2.train(x = predictors, y = response, training_frame = train, validation_frame = valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [12]:
final_rf_predictions = rf_v2.predict(test)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [13]:
final_rf_predictions

predict,p0,p1
0,0.941176,0.0588235
0,1,0
0,1,0
0,1,0
0,0.882353,0.117647
0,0.941176,0.0588235
0,1,0
0,1,0
0,0.941176,0.0588235
0,1,0


In [14]:
(final_rf_predictions['predict']==test['FraudFound_P']).as_data_frame(use_pandas=True).mean()

predict    0.921752
dtype: float64

In [15]:
rf_v2.confusion_matrix()

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2857142857142857: 


,0,1,Error,Rate
0,12187.0,853.0,0.0654,(853.0/13040.0)
1,552.0,285.0,0.6595,(552.0/837.0)
Total,12739.0,1138.0,0.1012,(1405.0/13877.0)


In [16]:
from h2o.grid.grid_search import H2OGridSearch

In [17]:
gbm_params1 = {'learn_rate': [0.01, 0.1],
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0]}